# **LP2_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met.  """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Parameters:

Let R1 be the first raw material and R2 be the second raw material.

c1 = Cost/unit of R1 
c2 = Cost/unit of R2 

s1 = Selling price/unit of drug 1 
s2 = Selling price/unit of drug 2 

a1 = Amount of active ingredient/unit in R1 
a2 =  Amount of active ingredient/unit in R2 

b = Budget for purchasing raw materials 
k1 = Capacity for producing drug 1 
k2 = Capacity for producing drug 2 

r1 = Requirement of active ingredient for drug 1 
r2 = Requirement of active ingredient for drug 2 

p1 = Operational cost for producing drug 1 
p2 = Operational cost for producing drug 2 

u1 = Storage capacity for R1
u2 = Storage capacity for R2

Note: a1 is allowed to vary by 0.5%; a2 by 2%.

Decision variables:

lr1 = quantity of R1 to buy 
lr2 = quantity of R2 to buy 

Objective function:

The objective is to maximize the net profit (income from sales - purchasing costs - operating costs)

max Z = s1*(a1*lr1/r1 - p1/k1) + s2*(a2*lr2/r2 - p2/k2 ) - c1*lr1 - c2*lr2

Subject to constrain

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is Python code using the Pyomo package:

```python
from pyomo.environ import *
import random

# Creating a model instance
model = ConcreteModel()

# Sample data
c1, c2 = random.uniform(1, 10), random.uniform(1, 10)
s1, s2 = random.uniform(10, 20), random.uniform(10, 20)
a1, a2 = random.uniform(0.8, 1), random.uniform(0.8, 1)
b = random.uniform(100, 200)
k1, k2 = random.uniform(50, 100), random.uniform(50, 100)
r1, r2 = random.uniform(0.1, 0.5), random.uniform(0.1, 0.5)
p1, p2 = random.uniform(1, 10), random.uniform(1, 10)
u1, u2 = random.uniform(50, 100), random.uniform(50, 100)

# Setting the decision variables
model.lr1 = Var(within=NonNegativeReals)
model.lr2 = Var(within=NonNegativeReals)

# Setting the objective
model.profit = Objective(expr=s1*(a1*model.lr1/r1 - p1/k1) + s2*(a2*model.lr2/r2 - p2/k2 ) - c1*model.lr1 - c2*model.lr2,
                         sense=maximize)

# Setting the constraints
model.budget_constraint = Constraint(expr=c1*model.lr1 + c2*model.lr2 <= b)
model.capacity1_constraint = Constraint(expr=a1*model.lr1/r1 <= k1)
model.capacity2_constraint = Constraint(expr=a2*model.lr2/r2 <= k2)
model.storage1_constraint = Constraint(expr=model.lr1 <= u1)
model.storage2_constraint = Constraint(expr=model.lr2 <= u2)
model.ingredient_requirement1_constraint = Constraint(expr=a1*(1-0.005)*model.lr1 >= r1)
model.ingredient_requirement2_constraint = Constraint(expr=a2*(1-0.02)*model.lr2 >= r2)

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# Printing the decision variables and the maximum profit
print('lr1:', value(model.lr1))
print('lr2:', value(model.lr2))
print('Profit:', value(model.profit))
```

In this code, we use sample numbers randomly generated within given intervals for all the parameters. For real world situation, we need to replace those with actual data points.

### **Run the code Generated by GPT4**

In [24]:
from pyomo.environ import *
import random

# Creating a model instance
model = ConcreteModel()

# Sample data - inputted by human, parameters that were not supposed to be a part of the solution were given random numbers.
s1, s2 = 6200, 6900

a1, a2  = 0.01 , 0.02
b = 100000
k1, k2 = 10000, 10000
r1, r2 = 0.5, 0.6
p1, p2 = 700, 800
u1, u2 = 500, 500


# Setting the decision variables
model.lr1 = Var(within=NonNegativeReals)
model.lr2 = Var(within=NonNegativeReals)

# Setting the objective
model.profit = Objective(expr=s1*(a1*model.lr1/r1 - p1/k1) + s2*(a2*model.lr2/r2 - p2/k2 ) - c1*model.lr1 - c2*model.lr2,
                         sense=maximize)

# Setting the constraints
model.budget_constraint = Constraint(expr=c1*model.lr1 + c2*model.lr2 <= b)
model.capacity1_constraint = Constraint(expr=a1*model.lr1/r1 <= k1)
model.capacity2_constraint = Constraint(expr=a2*model.lr2/r2 <= k2)
model.storage1_constraint = Constraint(expr=model.lr1 <= u1)
model.storage2_constraint = Constraint(expr=model.lr2 <= u2)
model.ingredient_requirement1_constraint = Constraint(expr=a1*(1-0.005)*model.lr1 >= r1)
model.ingredient_requirement2_constraint = Constraint(expr=a2*(1-0.02)*model.lr2 >= r2)

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# Printing the decision variables and the maximum profit
print('lr1:', value(model.lr1))
print('lr2:', value(model.lr2))
print('Profit:', value(model.profit))

lr1: 500.0
lr2: 500.0
Profit: 167631.19359157354


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**